# Significant Feature Enhancement

As part of the enemies group, we decided to make our significant feature enhancement a Final Boss level that will come after all levels.

This blog shows how we incorporated JavaScript Objects, Finite State Machines, and Single Responsibility Principle.


# JavaScript Objects

But first, what is a JavaScript Object?

To our knowledge, a JavaScript object is an object that has a collection of properties. These are referenced using key:value pairs.

In our boss level (and the whole game altogether), JavaScript objects get utilized quite frequently. They're used in enemies, players, game elements, and so much more!

We can see this when everything gets called to instantiate the level in GameSetup.js

```js
{ name: 'boss', id: 'boss', class: Boss, data: this.assets.enemies.boss, xPercentage: 0.5, minPosition: 0.3 },
```
Here, we are creating an object with multiple properties. Let's go step by step and see what each of them do.

 - name: human readable name for the object that shows up on the display
 - id: identifier used within the code of the game
 - class: defines behavior of the object (so how the boss enemy functions)
 - data: the actual spritesheet of the boss enemy
 - Xpercentage: horizontal position of object relative to canvas size
 - minPosition: minimum horizontal position the boss can move to (0.3 in this case)

Overall, we can see the importance of JavaScript objects within JS code and how it gets used to boil down a complex object to make everything more centralized and easier to work with.


# Finite State Machines
As per our understanding, a Finite State Machine (FSM) is a machine that can only be in one state out of a finite number of states. Without the fluff, all this means is that it can only be in one state at a time.

We used this when defining different rows and columns for our sprites when they were in different states. Using our Random Event, the Boss has to be in 1 of 7 states (finite number of states). 

Those were:
 - idleL
 - idleR
 - left
 - right
 - attackL
 - attackR
 - death

Each of these states had rows and columns assigned to them as per our spritesheet. These were all defined in GameSetup.js as follows.

```js
boss: {
	src: "/images/platformer/sprites/boss.png",
	width: 64,
	height: 64,
	scaleSize: 320,
	speedRatio: 0.2,
	animationSpeed: 6,
	idleL: { row: 9, frames: 0, idleFrame: { column: 1, frames: 0 } },
	idleR: { row: 11, frames: 0, idleFrame: { column: 1, frames: 0 } },
	left: { row: 9, frames: 8, idleFrame: { column: 7, frames: 0 } },
	right: { row: 11, frames: 8, idleFrame: { column: 7, frames: 0 } },
	attackL: { row: 13, frames: 5 },
	attackR: { row: 15, frames: 5 },
	death: { row: 20, frames: 5 },
},
```

Each state here has a specific action attached to it. For example, the idleR state has the following:
```js
{ row: 11, frames: 0, idleFrame: { column: 1, frames: 0 } },
```
This indicates the action for each state. The states for the boss get called in Boss.js where all the actions and behaviors for the Boss are defined.

First, it checks whether the state is dead or not and assigns accordingly.
```js
updateFrameX() {
// Update animation frameX of the object
	if(!this.death  ||  this.state.animation  !=  "death"){
		if (this.frameX  <  this.maxFrame) {
			if(this.counter  >  0){
				this.frameX  =  this.frameX;
				this.counter--;
			}
			else {
				this.frameX++
				this.counter  =  this.animationSpeed;
			}
		} else {
			this.frameX  =  this.minFrame;
		}
	}
	else  if(this.death  &&  this.state.animation  ==  "death"){
		this.animationSpeed  =  50;
		if (this.frameX  <  this.maxFrame) {
			if(this.counter  >  0){
				this.frameX  =  this.frameX;
				this.counter--;
			}
			else{
				this.frameX++
				this.counter  =  this.animationSpeed;
			}
		} else {
			this.destroy();
		}
	}
}
```

We then check for the other states in the randomEvent method still in Boss.js.

```js
randomEvent(){
	if (GameControl.randomEventId  ===  1  &&  GameControl.randomEventState  ===  2){ //event: stop the zombie
		this.state.direction  =  "left";
		this.state.animation  =  "idleL";
		GameControl.endRandomEvent();
	}
	else if (GameControl.randomEventId  ===  2  &&  GameControl.randomEventState  ===  2){ //event: stop the zombie
		this.state.direction  =  "right";
		this.state.animation  =  "idleR";
		GameControl.endRandomEvent();
	}
	else if (GameControl.randomEventId  ===  3  &&  GameControl.randomEventState  ===  2){ //event: stop the zombie
		this.state.direction  =  "left";
		this.state.animation  =  "left";
		GameControl.endRandomEvent();
	}
	else if (GameControl.randomEventId  ===  4  &&  GameControl.randomEventState  ===  2){ //event: stop the zombie
		this.state.direction  =  "right";
		this.state.animation  =  "right";
		GameControl.endRandomEvent();
	}
}
```

We also see Finite State Machines applied in Enemy.js.

```js
initEnvironmentState = {
        // Enemy
        animation: 'right', //current animation
        direction: 'right', //facing direction, only contain left or right
        isDying: false,
};

this.state = {...this.initEnvironmentState}; // Enemy and environment states 
```
*In Enemy.js*

Just like the player, we can set up a Finite State Machine to the Enemy. We create a variable/object called 'initEnvironmentState' that contains a list of variables and in this case, are 'animation', 'direction', and 'isDying".

Inside the constructor, we set up a property called "this.state" and assign 'initEnvironmentState' to it.

This is done so that when we call 'this.state.animation', it will return the current animation of the enemy. This is the same as 'this.state.direction', and 'this.state.isDying'.


Overall, it helps us to organize the property such that it can only have one state, and when each state gets called / assigned, a new action occurs.


# Single Responsibility Principle
This is the principle that every function, every method should have only one function.

Let's examine how refactoring a method to adhere to SRP can simplify code maintenance and improve flexibility, especially when extending a class. Here's the original code snippet with a complex ```update()``` method:

```js
update() {
    super.update();
    this.setAnimation(this.state.animation);

    // Check for boundaries
    if (this.x <= this.minPosition || (this.x + this.canvasWidth >= this.maxPosition)) {
        if (this.state.direction === "left") {
            this.state.animation = "right";
            this.state.direction = "right";
        } else if (this.state.direction === "right") {
            this.state.animation = "left";
            this.state.direction = "left";
        }
    }

    // Update movement
    if (this.state.animation === "right") {
        this.speed = Math.abs(this.speed);
    } else if (this.state.animation === "left") {
        this.speed = -Math.abs(this.speed);
    } else if (this.state.animation === "idle") {
        this.speed = 0;
    } else if (this.state.animation === "death") {
        this.speed = 0;
    }

    // Move the enemy
    this.x += this.speed;

    this.playerBottomCollision = false;
}
```

This code does many things at once: setting animations, checking boundaries, and updating movement. To improve it, we can extract specific responsibilities into dedicated methods, as shown below:

```js
checkBoundaries() {
    if (this.x <= this.minPosition || (this.x + this.canvasWidth >= this.maxPosition)) {
        if (this.state.direction === "left") {
            this.state.animation = "right";
            this.state.direction = "right";
        } else if (this.state.direction === "right") {
            this.state.animation = "left";
            this.state.direction = "left";
        }
    }
}

updateMovement() {
    if (this.state.animation === "right") {
        this.speed = Math.abs(this.speed);
    } else if (this.state.animation === "left") {
        this.speed = -Math.abs(this.speed);
    } else {
        this.speed = 0;
    }

    // Move the enemy
    this.x += this.speed;

    this.playerBottomCollision = false;
}

update() {
    super.update();
    this.setAnimation(this.state.animation);

    this.checkBoundaries();
    this.updateMovement();
}
```

Now, each method has a single responsibility, making it easier to read, understand, and maintain. This refactoring provides a few key benefits:

- Clarity: The code is easier to understand since each method does one thing. This makes it simpler to follow the logic and identify specific responsibilities.

- Modularity: By isolating functionality into separate methods, it's easier to change or extend specific behavior without affecting the rest of the class.

- Extensibility: If you're creating a subclass that extends Enemy, you can easily override individual methods like updateMovement() to change behavior without needing to modify the original class. This flexibility is crucial in object-oriented programming, allowing for specialized behavior in subclasses.

For example, if we have a new Enemy that extends the Enemy class, and we want it to have a new movement instead of using the old one.
We can just overwrite the method "updateMovement()" by creating a function that has the same name called "updateMovement()" and just put the code we want to overwrite into the new function.

We don't need to go back and change the Enemy class such as adding an if-statement to the code inside the update() function that manages the movement.
